# Clock Game Sync Word Finder

In [50]:
Numbers = [
    'dodici',
    'uno',
    'due',
    'tre',
    'quattro',
    'cinque',
    'sei',
    'sette',
    'otto',
    'nove',
    'dieci',
    'undici',
]

nNumbers = len(Numbers)

In [51]:
#Automaton for Italian Regions
Automaton = [
    [7, 11],
    [5, 3],
    [1, 3],
    [6, 0],
    [5, 1],
    [7, 6],
    [0, 7],
    [8, 11],
    [9, 5],
    [5, 4],
    [9, 8],
    [10, 8],
]

In [52]:
Automaton = [[len(n) % nNumbers, (len(n) + i) % nNumbers] for i, n in enumerate(Numbers)]
Automaton

[[6, 6],
 [3, 4],
 [3, 5],
 [3, 6],
 [7, 11],
 [6, 11],
 [3, 9],
 [5, 0],
 [4, 0],
 [4, 1],
 [5, 3],
 [6, 5]]

In [54]:
def Delta(state, move):
    return Automaton[state][move]

In [55]:
def Execute(word, starting_state):
    state = starting_state
    for c in word:
        state = Delta(state, int(c))
    return state

In [56]:
def hasKnownSyncWord(SyncWords, word):
    for w in SyncWords:
        if w in word:
            return True
    return False

In [62]:
maxLen = 15
iter = 2**maxLen
SyncWords = []

for n in range(iter):
    word = str(format(n + 2, 'b'))[1:]
    if not hasKnownSyncWord(SyncWords, word):
        outSet = set()

        for i in range(nNumbers):
            output = Execute(word, i)
            outSet.add(output)
            
        if(len(outSet) == 1):
            SyncWords.append(word)
            print(n, word.replace('0', 'A').replace('1', 'B') + ' ->', outSet)

61 BBBBB -> {3}
89 ABBABB -> {3}
127 AAAAAAB -> {6}
269 AAAABBBB -> {3}
441 BABBBABB -> {3}
519 AAAAABAAB -> {3}
825 BAABBBABB -> {3}
831 BABAAAAAB -> {6}
959 BBBAAAAAB -> {6}
1001 BBBBABABB -> {3}
1449 ABBABABABB -> {3}
1599 BAABAAAAAB -> {6}
1677 BABAAABBBB -> {3}
1737 BABBAABABB -> {3}
1758 BABBBAAAAA -> {11}
1761 BABBBAAABB -> {3}
1933 BBBAAABBBB -> {3}
1993 BBBBAABABB -> {3}
2121 AAAABAABABB -> {3}
2142 AAAABBAAAAA -> {11}
2145 AAAABBAAABB -> {3}
2889 ABBABAABABB -> {3}
3213 BAABAAABBBB -> {3}
3273 BAABBAABABB -> {3}
3294 BAABBBAAAAA -> {11}
3297 BAABBBAAABB -> {3}
3335 BABAAAABAAB -> {3}
3465 BABBAAABABB -> {3}
3847 BBBAAAABAAB -> {3}
3977 BBBBAAABABB -> {3}
3998 BBBBABAAAAA -> {11}
4001 BBBBABAAABB -> {3}
4158 AAAAABAAAAAA -> {11}
4161 AAAAABAAAABB -> {3}
4163 AAAAABAAABAB -> {6}
4177 AAAAABABAABB -> {3}
4233 AAAABAAABABB -> {3}
4329 AAAABBBABABB -> {3}
5689 ABBAAABBBABB -> {3}
5769 ABBABAAABABB -> {3}
5790 ABBABABAAAAA -> {11}
5793 ABBABABAAABB -> {3}
6407 BAABAAAABAAB -> {3}
6

In [58]:
def Step(State, move):
    res = set()
    for s in State:
        res.add(Automaton[s][move])
    return tuple(sorted(res))

def Neighbors(State):
    output = []
    for m in range(len(Automaton[0])):
        output.append(Step(State, m))
    return output

def minOpen(Open, FScores):
    minScore = FScores[Open[0]]
    out = Open[0]
    for o in Open[1:]:
        score = FScores[o]
        if score < minScore:
            minScore = score
            out = o
    return out

def BuildPath(CameFrom, node):
    if node in CameFrom:
        return BuildPath(CameFrom, CameFrom[node][1]) + str(CameFrom[node][0])
    else:
        return ""


In [59]:
def H_Dis(state):
    return len(state)

def G_dis(state1, state2):
    return 1

In [60]:
def ShortestWord():
    InitialState = tuple([_ for _ in range(nNumbers)])
    steps = 0

    Open = [InitialState]
    Closed = []

    GScores = {InitialState: 0}
    HScores = {InitialState: H_Dis(InitialState)}
    FScores = {InitialState: HScores[InitialState]}

    cameFrom = {}

    while len(Open) != 0:
        current = minOpen(Open, FScores)
        steps += 1

        if len(current) == 1:
            return BuildPath(cameFrom, current), current, steps
            
        Open.remove(current)
        Closed.append(current)

        for i, n in enumerate(Neighbors(current)):
            if n in Closed:
                continue

            tentativeGScore = GScores[current] + G_dis(current, n)

            if not n in Open:
                Open.append(n)
                betterTentative = True
            elif tentativeGScore < GScores[n]:
                betterTentative = True
            else:
                betterTentative = False

            if betterTentative:
                cameFrom[n] = (i, current)

                GScores[n] = tentativeGScore
                HScores[n] = H_Dis(n)
                FScores[n] = GScores[n] + HScores[n]
    return 'Failure'

word, end, steps = ShortestWord()
word = word.replace('0', 'A').replace('1', 'B')
print(steps, word, '->', end)


10 BBAAAB -> (9,)
